### Une approche purement supervisée

Lorsqu'il s'agit de résoudre un problème de classification supervisée, le choix du modèle approprié revêt une importance cruciale pour obtenir des performances optimales. 
Dans le cadre de notre étude sur la prédiction des tags, on a sélectionné quelques algorithmes de classification populaires, chacun avec ses propres caractéristiques et avantages.

- **SGD Classifier (Stochastic Gradient Descent)** :
    Le SGD Classifier est un modèle linéaire entraîné à l'aide de la descente de gradient stochastique. Il est efficace pour traiter de grands ensembles de données et est souvent utilisé dans des scénarios où le nombre de fonctionnalités est élevé. Il peut être utilisé pour une variété de tâches de classification et est particulièrement utile lorsque le temps de formation est un facteur critique.

- **Logistic Regression** :
    La régression logistique est un modèle linéaire utilisé pour la classification binaire et multiclasse. Malgré son nom, il est principalement utilisé pour la classification plutôt que pour la régression. Il fournit des probabilités de classe et est interprétable, ce qui le rend utile pour comprendre l'importance des fonctionnalités.

- **Multinomial Naive Bayes (MultinomialNB)** :
    Le modèle Naive Bayes multinomial est basé sur le théorème de Bayes avec une supposition naive d'indépendance entre les fonctionnalités. Il est couramment utilisé pour la classification de texte, en particulier pour les tâches où les fonctionnalités sont des comptages d'occurrences de mots.

- **Linear Support Vector Classifier (LinearSVC)** :
    Le LinearSVC est une implémentation de SVM (Support Vector Machine) linéaire, qui est un modèle de classification puissant. Il vise à trouver l'hyperplan qui sépare au mieux les classes dans l'espace des fonctionnalités. Le LinearSVC est efficace pour les ensembles de données de grande taille et à haute dimensionnalité.




Justification du Choix

 - SGDClassifier : Choisi pour sa flexibilité et sa capacité à traiter de grands ensembles de données.
 - Logistic Regression : Sert de baseline robuste et est souvent performant.
 - Multinomial Naive Bayes : Idéal pour les données textuelles, rapide et efficace.
 - Linear SVC : Robuste et performant, particulièrement adapté aux tâches de classification de grande dimension.



### Méthodes d’évaluation



- **Jaccard Score** :
    Le score Jaccard est une mesure de similarité entre deux ensembles. Dans le contexte de la classification multi-labels, il mesure la similarité entre les ensembles de vrais et de prédictions étiquetés. Il est calculé en divisant la taille de l'intersection des ensembles par la taille de leur union. 
  => Plus le score Jaccard est proche de 1, plus les ensembles sont similaires.

- **Hamming Loss** :
    La perte de Hamming mesure la fraction des étiquettes mal prédites par rapport au nombre total d'étiquettes. Elle est calculée en prenant la moyenne des erreurs de prédiction pour chaque échantillon. 
  => Une Hamming loss de 0 indique une prédiction parfaite, et une Hamming loss de 1 indique une prédiction complètement incorrecte.

- **Accuracy (Précision)** :
    L'exactitude mesure la proportion d'étiquettes correctement prédites par rapport au nombre total d'étiquettes. C'est une mesure globale de la performance du modèle. 
  => Une précision de 1 indique une prédiction parfaite, tandis qu'une précision de 0 indique une prédiction complètement incorrecte.

- **Precision (Précision)** :
    La précision mesure la proportion d'étiquettes prédites comme positives qui sont réellement positives. Elle est calculée en divisant le nombre de vrais positifs par la somme des vrais positifs et des faux positifs. 
  => Une précision élevée indique un faible taux de faux positifs.

- **Recall (Rappel)** :
    Le rappel mesure la proportion d'étiquettes positives réellement prédites par rapport à toutes les étiquettes positives. Il est calculé en divisant le nombre de vrais positifs par la somme des vrais positifs et des faux négatifs. 
  => Un rappel élevé indique un faible taux de faux négatifs.

Ces métriques d'évaluation sont importantes pour évaluer la performance des modèles de classification multi-labels. 

###### imports

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import hamming_loss, accuracy_score, precision_score, recall_score, f1_score

In [2]:
import warnings

# Ignorer tous les avertissements
warnings.filterwarnings("ignore")


In [3]:
data = pd.read_csv(f'data_final.csv')

In [4]:
# Supprimer la colonne Unnamed: 0
data = data.drop(columns=['Unnamed: 0'])

In [5]:
data.head(3)

,Question,Tags
0,fastest way get value looking fastest way obta...,performance algorithm
1,use socket api issue getting socket api work p...,c++ c
2,net xml comment api documentation easy way pro...,visual-studio


In [6]:
data['Tags'] = data['Tags'].str.split()

In [7]:
# Prendre un échantillon aléatoire de 2000 lignes du DataFrame
sample_data = data.sample(n=2000, random_state=42)

In [8]:
X = sample_data['Question']
y = sample_data['Tags']

In [9]:
y.iloc[0]

['database', 'algorithm']

In [10]:
multilabel_binarizer = MultiLabelBinarizer()
y_bin = multilabel_binarizer.fit_transform(y)

In [11]:
y_bin[:2]

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0]])

In [12]:
[(index_label, yy) for (index_label, yy) in enumerate(y_bin[0]) if yy==1]

[(0, 1), (11, 1)]

In [13]:
classes = multilabel_binarizer.classes_
print(classes)
len(classes)

['algorithm' 'amazon-web-services' 'android' 'angular' 'arrays' 'asp.net'
 'asp.net-mvc' 'c' 'c#' 'c++' 'css' 'database' 'django' 'docker' 'gcc'
 'html' 'ios' 'iphone' 'java' 'javascript' 'jquery' 'json' 'linux' 'macos'
 'multithreading' 'mysql' 'node.js' 'objective-c' 'pandas' 'performance'
 'php' 'postgresql' 'python' 'r' 'reactjs' 'ruby' 'ruby-on-rails' 'spring'
 'spring-boot' 'sql' 'sql-server' 'string' 'swift' 'typescript'
 'unit-testing' 'visual-studio' 'windows' 'xcode']


48

In [14]:
vectorizer_X = TfidfVectorizer(analyzer = 'word',
                                       min_df=0.0,
                                       max_df = 1.0,
                                       strip_accents = None,
                                       encoding = 'utf-8', 
                                       preprocessor=None,
                                       token_pattern=r"(?u)\S\S+",
                                       max_features=1000)


In [15]:
X_tfidf = vectorizer_X.fit_transform(X)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y_bin, test_size = 0.2, random_state = 0) # Do 80/20 split

In [17]:
y_test

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

Pour évaluer les modèles, on va utiliser le score Jaccard car il est le mieux adapté à la classification multi labels

In [18]:
def jacard_score(y_true,y_pred):
    jacard = np.minimum(y_true,y_pred).sum(axis=1) / np.maximum(y_true,y_pred).sum(axis=1)
    return jacard.mean()*100

In [19]:
def calculate_results(y_pred, y_test, model):
    jaccard = jacard_score(y_test, y_pred)
    hamming = hamming_loss(y_test, y_pred) 
    accuracy = accuracy_score(y_test, y_pred) 
    precision = precision_score(y_test, y_pred, average='samples')
    recall = recall_score(y_test, y_pred, average='samples') 
    f1 = f1_score(y_test, y_pred, average='macro')
    return jaccard, hamming, accuracy, precision, recall, f1


In [20]:
sgd = SGDClassifier()
lr = LogisticRegression()
mn = MultinomialNB()
svc = LinearSVC()


In [21]:
# Définir les classificateurs à tester
classifiers = [SGDClassifier(), LogisticRegression(), MultinomialNB(), LinearSVC()]

# Initialiser une liste pour stocker les résultats
results_list = []

# Définir les noms des modèles
model_names = ['SGD', 'Logistic Regression', 'Multinomial NB', 'Linear SVC']

# Définir les méthodes d'évaluation
evaluation_methods = ['Jaccard score', 'Hamming loss', 'Accuracy', 'Precision', 'Recall', 'f1']

# Boucler sur chaque classificateur
for classifier, model_name in zip(classifiers, model_names):
    # Créer le modèle OneVsRestClassifier
    model = OneVsRestClassifier(classifier)
    
    # Entraîner le modèle
    model.fit(X_train, y_train)
    
    # Faire des prédictions
    y_pred = model.predict(X_test)
    
    # Calculer les résultats
    jaccard, hamming, accuracy, precision, recall, f1 = calculate_results(y_pred, y_test, model)
    
    # Ajouter les résultats à la liste
    results_list.append([model_name, jaccard, hamming, accuracy, precision, recall, f1])

# Convertir la liste en DataFrame
results_df = pd.DataFrame(results_list, columns=['Modèle'] + evaluation_methods)

# Transposer le DataFrame pour avoir les modèles en colonnes et les méthodes d'évaluation en lignes
results_df = results_df.set_index('Modèle').T


In [22]:
results_df

Modèle,SGD,Logistic Regression,Multinomial NB,Linear SVC
Jaccard score,37.185714,10.629167,6.837500,35.004167
Hamming loss,0.029740,0.032344,0.033177,0.026979
Accuracy,0.205000,0.080000,0.045000,0.237500
Precision,0.473542,0.136250,0.090417,0.447292
Recall,0.442458,0.106417,0.072958,0.374958
f1,0.409155,0.053116,0.052672,0.372037


Le modèle SGD obtient le meilleur Jaccard Score, Hamming Loss, Precision, Recall et F1-score parmi tous les modèles

**SGD** semble être un bon choix global, car il offre des performances globalement meilleures.

##### SGDClassifier

In [23]:
from sklearn.model_selection import GridSearchCV

# Définir les hyperparamètres à rechercher
param_grid = {
    'estimator__alpha': [0.0001, 0.001, 0.01],
    'estimator__loss': ['hinge', 'log', 'modified_huber'],
    'estimator__penalty': ['l1', 'l2', 'elasticnet'],
    'estimator__learning_rate': ['optimal', 'constant', 'adaptive']
}

# Créer une instance de GridSearchCV
grid_search = GridSearchCV(
    estimator=OneVsRestClassifier(SGDClassifier()),
    param_grid=param_grid,
    scoring='accuracy',  # Choisissez la métrique de performance appropriée
    cv=5  # Nombre de folds pour la validation croisée
)

# Ajuster GridSearchCV sur les données d'entraînement
grid_search.fit(X_train, y_train)




GridSearchCV(cv=5, estimator=OneVsRestClassifier(estimator=SGDClassifier()),
             param_grid={'estimator__alpha': [0.0001, 0.001, 0.01],
                         'estimator__learning_rate': ['optimal', 'constant',
                                                      'adaptive'],
                         'estimator__loss': ['hinge', 'log', 'modified_huber'],
                         'estimator__penalty': ['l1', 'l2', 'elasticnet']},
             scoring='accuracy')

In [24]:
# Afficher les meilleurs paramètres et le score correspondant
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

Best Parameters: {'estimator__alpha': 0.001, 'estimator__learning_rate': 'optimal', 'estimator__loss': 'modified_huber', 'estimator__penalty': 'l1'}
Best Score: 0.24437499999999995


In [25]:
# Utiliser les meilleurs paramètres trouvés
best_params = grid_search.best_params_

# Créer une instance de SGDClassifier avec les meilleurs paramètres
sgd_classifier = SGDClassifier(alpha=best_params['estimator__alpha'], loss=best_params['estimator__loss'],
                              penalty=best_params['estimator__penalty'], 
                              learning_rate=best_params['estimator__learning_rate'])

# Créer une instance de OneVsRestClassifier avec le SGDClassifier optimisé
final_model = OneVsRestClassifier(sgd_classifier)

# Ajuster le modèle final sur les données d'entraînement
final_model.fit(X_train, y_train)

# Faire des prédictions sur les données de test
y_pred = final_model.predict(X_test)


In [26]:
accuracy = accuracy_score(y_test, y_pred) 
print(accuracy)

0.27


In [27]:
from joblib import dump, load

# Save the model
dump(final_model, 'model_sgd.joblib')
dump(vectorizer_X, 'vectorizer_X.joblib')
dump(multilabel_binarizer, 'multilabel_binarizer.joblib')

['multilabel_binarizer.joblib']

In [28]:
# Load the model
loaded_model = load('model_sgd.joblib')
loaded_vectorizer = load('vectorizer_X.joblib')


In [29]:
X_test.shape

(400, 1000)

Test

In [38]:
x = loaded_vectorizer.transform(["How can I effectively integrate Python and Cpp in a single application to leverage the strengths of both languages?I am working on a project where I need to combine the rapid development capabilities of Python with the performance efficiency of Cpp"])
x

<1x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 9 stored elements in Compressed Sparse Row format>

In [39]:
predictions = loaded_model.predict(x)

In [40]:
predictions


array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0]])

In [41]:
predicted_labels = [classes[i] for i, prediction in enumerate(predictions[0]) if prediction == 1]

print("Predicted labels:", predicted_labels)


Predicted labels: ['c++', 'python']
